![](https://i.imgur.com/qkg2E2D.png)

# UnSupervised Learning Methods

## Exercise 004 - Part III

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 1.0.000 | 08/09/2023 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/UnSupervisedLearningMethods/2023_08/Exercise0004Part003.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning
from sklearn.cluster import KMeans
from sklearn.datasets import make_s_curve, make_swiss_roll
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import kneighbors_graph

# Computer Vision

# Miscellaneous
import os
import math
from platform import python_version
import random
import time
import urllib.request

# Typing
from typing import Callable, List, Tuple, Union

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatSlider, interact, IntSlider, Layout


## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

In [ ]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

DATA_FILE_URL   = r'https://drive.google.com/uc?export=download&confirm=9iBg&id=1lT6eWVRvfx_iusI9zB1fLg_g64n_141A'
DATA_FILE_NAME  = r'Rings5.mat'

TOTAL_RUN_TIME = 10 #<! Don't touch it!


In [ ]:
# Auxiliary Functions

def Plot3DScatter(mX: np.ndarray, hA: plt.Axes, vC: np.ndarray = None) -> None:
    m = mX.min()
    M = mX.max()
    if vC is not None:
        hA.scatter(*mX.T, s = 50,  c = vC, edgecolor = 'k', alpha = 1)
    else:
        hA.scatter(*mX.T, s = 50,  c = 'lime', edgecolor = 'k', alpha = 1)
    hA.set_xlim([m, M])
    hA.set_ylim([m, M])
    hA.set_zlim([m, M])
    hA.set_xlabel('$x_1$')
    hA.set_ylabel('$x_2$')
    hA.set_zlabel('$x_3$')

def Plot2DScatter(mX: np.ndarray, hA: plt.Axes, vC: np.ndarray = None) -> None:
    m = mX.min()
    M = mX.max()
    if vC is not None:
        hA.scatter(*mX.T, s = 50,  c = vC, edgecolor = 'k', alpha = 1)
    else:
        hA.scatter(*mX.T, s = 50,  c = 'lime', edgecolor = 'k', alpha = 1)
    hA.set_xlim([m, M])
    hA.set_ylim([m, M])
    hA.set_xlabel('$x_1$')
    hA.set_ylabel('$x_2$')


def MaximumSparseMat(mA: sp.sparse.spmatrix, mB: sp.sparse.spmatrix) -> sp.sparse.spmatrix:
    """
    Returns the element wise maximum of sparse matrices `mA` and `mB`.
    """
    mAgtB = (mA > mB).astype(int)
    mM = mAgtB.multiply(mA - mB) + mB

    return mM


## Guidelines

 - Fill the full names and ID's of the team members in the `Team Members` section.
 - Answer all questions / tasks within the Jupyter Notebook.
 - Use MarkDown + MathJaX + Code to answer.
 - Verify the rendering on VS Code.
 - Don't use `pip install` on the submitted notebook!  
   If you need a package that is not imported above use **the dedicated cell**.  
   Comment what do you need the package for and the version needed.
 - If you need functions from previous notebook copy them into a file called `AuxFun.py`.  
   Import the function in the dedicated cell.
 - Submission in groups (Single submission per group).
 - The submission files should have the format: `<fileName>_GRP_<#>`.  
   For instance, `Exercise001Part002_GRP_A.ipynb` or `AuxFun_GRP_A.py`.
 - You may and _should_ use the forums for questions.
 - Good Luck!

<font color='red'>Total run time must be **less than `TOTAL_RUN_TIME` seconds**</font>.

In [ ]:
# Run Time
print(f'The total run time must not exceed: {TOTAL_RUN_TIME} [Sec]')
startTime = time.time()

The total run time must not exceed: 10 [Sec]


* <font color='brown'>(**#**)</font> The `Import Packages` section above imports most needed tools to apply the work. Please use it.
* <font color='brown'>(**#**)</font> You may replace the suggested functions to use with functions from other packages.
* <font color='brown'>(**#**)</font> Whatever not said explicitly to implement maybe used by a 3rd party packages.

## Team Members

- `Matthew_koton_806614`
- `Aviv_Ples_318357233`

In [ ]:
# Students Packages to Import
# If you need a package not listed above, use this cell
# Do not use `pip install` in the submitted notebook
from CourseAuxFun import *
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

ModuleNotFoundError: ignored

## Generate / Load Data

In [ ]:
# Download Data
# This section downloads data from the given URL if needed.

if (DATA_FILE_NAME != 'None') and (not os.path.exists(DATA_FILE_NAME)):
    urllib.request.urlretrieve(DATA_FILE_URL, DATA_FILE_NAME)

## 8. IsoMap & Laplacian EigenMaps

### 8.1. IsoMap Algorithm

In this section we'll implement a SciKit Learn API compatible class for the IsoMap algorithm.  
For the graphs we'll use the _K Nearest Neighbors_ approach.

The class should implement the following methods:

1. `__init____()` - The object constructor by the encoder dimension.  
2. `fit()` - Given a data set builds the encoder.  
3. `transform()` - Applies the encoding on the input data in out of sample manner.  
4. `fix_transform()` - Given a data set builds the encoder and applies the encoding.  

* <font color='brown'>(**#**)</font> Pay attention to data structure (`Nx x Nx` / `Nx x Ny`).
* <font color='brown'>(**#**)</font> Do not use any loops in you implementation.
* <font color='brown'>(**#**)</font> You should use your `CMDS()` implementation.
* <font color='brown'>(**#**)</font> Use `from CourseAuxFun.py import *` to import your code.
* <font color='brown'>(**#**)</font> You should use SciKit Learn's `sklearn.neighbors.kneighbors_graph`. Pay attention the output is a sparse matrix.
* <font color='brown'>(**#**)</font> You should use SciPy's `scipy.sparse.csgraph.connected_components` to check the graph is valid (Connected).  
* <font color='brown'>(**#**)</font> You should use SciPy's `scipy.sparse.csgraph.shortest_path` to computer the pairs shortest path matrix.  
* <font color='brown'>(**#**)</font> For the `transform()` methods you should use SciKit Learn's `sklearn.neighbors.NearestNeighbors`.  


In [ ]:

class IsoMap():
    def __init__(self, d: int = 2, k: int = 9):
        '''
        Constructing the object.
        Args:
            d - Number of dimensions of the encoder output.
            k - Number of neighbors in the graph construction.
        '''
        #===========================Fill This===========================#
        # 1. Keep the model parameters.


        self.d = d
        self.k = k
        self.mds = CMDS(d)
        #===============================================================#

    def fit(self, mX: np.ndarray):
        '''
        Fitting model parameters to the input.
        Args:
            mX - Input data with shape N x D.
        Output:
            self
        '''
        #===========================Fill This===========================#
        # 1. Build the graph from the data.
        # 2. Verify the graph is connected (Raise error if not).
        # 3. Build the encoder.
        # !! Use the K-NN method to build the graph.
        # !! Make sure the graph obeys the assumptions made.
        # !! The encoder should be based on the CMDS() class.

        self.mX = mX
        self.knn_graph_csr = kneighbors_graph(mX, self.k)

        num_components, labels = sp.sparse.csgraph.connected_components(self.knn_graph_csr)

        if num_components > 1:
            raise ValueError("The graph is not connected")


        self.shortest_path_graph = sp.sparse.csgraph.shortest_path(self.knn_graph_csr, method='auto')

        self.mds.fit(self.shortest_path_graph)
        #===============================================================#
        return self

    def transform(self, mY: np.ndarray) -> np.ndarray:
        '''
        Applies (Out of sample) encoding.
        Args:
            mY - Input data (Out of Sample) with shape N x D.
        Output:
            mZ - Low dimensional representation (embeddings) with shape N x d.
        '''
        #===========================Fill This===========================#
        # 1. Encode data using the model encoder.

        combined_matrix = np.vstack((self.mX, mY))
        knn_graph_combined_arr = kneighbors_graph(combined_matrix, self.k).toarray()


        knn = NearestNeighbors(n_neighbors=self.k)
        knn.fit(self.mX)
        distances, indices = knn.kneighbors(mY)
        for num, i in enumerate(indices):
            temp = np.zeros((self.mX.shape[0]+mY.shape[0]))
            temp[i]=1

#             print(self.mX.shape[0] + num)
#             print(knn_graph_combined_arr[self.mX.shape[0] + num].shape)
#             print(temp.shape)

            knn_graph_combined_arr[self.mX.shape[0] + num] = temp


        knn_graph_combined_csr = csr_matrix(knn_graph_combined_arr)
        shortest_distance_graph = sp.sparse.csgraph.shortest_path(knn_graph_combined_csr, method='auto')

        Dxy = shortest_distance_graph[:self.mX.shape[0], mX.shape[0]:]
        Dxx = sp.sparse.csgraph.shortest_path(self.knn_graph_csr, method='auto')

        N_ones = np.ones((self.mX.shape[0], mY.shape[0]))

        J = np.identity(self.mX.shape[0]) - np.ones((self.mX.shape[0], self.mX.shape[0])) / self.mX.shape[0]
        t = Dxy - (Dxx)@N_ones / self.mX.shape[0]

        kxy = -0.5 * np.dot(J ,t)

        mZ = self.mds.transform(kxy)
        #===============================================================#

        return mZ

    def fit_transform(self, mX: np.ndarray) -> np.ndarray:
        '''
        Applies encoding on the input.
        Args:
            mX - Input data (Distance matrix) with shape N x D.
        Output:
            mZ - Low dimensional representation (embeddings) with shape N x d.
        '''
        #===========================Fill This===========================#
        # 1. Apply the `fit()` method.
        # 2. Encode data using the model encoder.

        self.fit(mX)
        mZ = self.mds.transform(self.shortest_path_graph)
        #===============================================================#

        return mZ


* <font color='red'>(**?**)</font> Will `fit()` and then `transform()` will match the result of `fit_transform()`?  
  Make sure you understand this before proceeding.

### 8.2. Laplacian EigenMaps Algorithm

In this section we'll implement a SciKit Learn API compatible class for the Laplacian EigenMaps.  

The class should implement the following methods:

1. `__init____()` - The object constructor by the encoder dimension.  
2. `fit()` - Given a data set ($\boldsymbol{D}_{xx}$) builds the encoder.    
4. `fit_transform()` - Given a data set ($\boldsymbol{D}_{xx}$) builds the encoder and applies the encoding.  

* <font color='brown'>(**#**)</font> Pay attention to data structure (`Nx x Nx` / `Nx x Ny`).
* <font color='brown'>(**#**)</font> Do not use any loops in you implementation beside the main MM loop.
* <font color='brown'>(**#**)</font> Think about the difference in `transform()` and `fit_transform()` compared to `CMDS()` above.

In [ ]:
class LaplacianEigenMaps():
    def __init__(self, d: int = 2, k: int = 9, σ: float = 1):
        '''
        Constructing the object.
        Args:
            d - Number of dimensions of the encoder output.
            k - Number of neighbors in the graph construction.
            σ - The factor to multiply the median distance by.
        '''
        #===========================Fill This===========================#
        # 1. Keep the model parameters.

        self.d = d
        self.k = k
        self.sigma = σ
        #===============================================================#

    def fit(self, mX: np.ndarray):
        '''
        Fitting model parameters to the input.
        Args:
            mX - Input data with shape N x D.
        Output:
            self
        '''
        #===========================Fill This===========================#
        # 1. Build a valid graph.
        # 2. Calculate the number of connected components in the graph.
        # 3. Keep the parameters in the object.
        # !! Raise error if the graph has more than 1 connected component.



        knn_graph = kneighbors_graph(mX, self.k)

        ##num_components, labels = sp.sparse.csgraph.connected_components(self.knn_graph_csr)
        num_components, labels = sp.sparse.csgraph.connected_components(knn_graph)
        if num_components > 1:
            raise ValueError("graph not connected")

        self.mX = mX
        self.knn_graph = knn_graph


        #===============================================================#
        return self

    def fit_transform(self, mX: np.ndarray) -> np.ndarray:
        '''
        Applies encoding on input data.
        Args:
            mX - Input data (Distance matrix) with shape N x D.
        Output:
            mZ - Low dimensional representation (embeddings) with shape N x d.
        '''
        #===========================Fill This===========================#
        # 1. Apply the `fit()` method.
        # 2. Build the distance matrix.
        # 3. Set σ^2 to be the median squared euclidean distance multiplied by `self.σ^2``.
        # 4. Build the Affinity Matrix using Gaussian Weights.
        # 5. Build the Laplacian.
        # 6. Apply eigen decomposition to the Laplacian.
        # 7. Choose the eigen vectors wisely.
        # 8. Encode data.
        # !! You should chose the vectors

        self.fit(mX)
        ##sigma_squared = (np.median(mX) ** 2) * (self.sigma ** 2)
        sigma_squared = self.sigma ** 2


        knn_graph = self.knn_graph.toarray()

        distance_matrix = sp.spatial.distance.squareform(sp.spatial.distance.pdist(self.mX, 'euclidean'))
        N = distance_matrix.shape[0]
        weight_matrix = np.zeros((N, N))

        for i in range(N):
            neighbors = np.where(knn_graph[i] == 1)[0]
            for j in neighbors:
                weight_matrix[i, j] = np.exp(-distance_matrix[i, j] ** 2 / (2 * sigma_squared))
                weight_matrix[j, i] = weight_matrix[i, j]

        degree_matrix = np.diag(np.sum(weight_matrix, axis=1))
        laplacian_matrix = degree_matrix - weight_matrix

        eigvals, eigvecs = np.linalg.eigh(laplacian_matrix)

        sorted_indices = np.argsort(eigvals)[1:self.d+1]
        chosen_eigvecs = eigvecs[:, sorted_indices]

        mZ = chosen_eigvecs
        #===============================================================#

        return mZ


* <font color='red'>(**?**)</font> Why is the `transform()` method not asked to be implemented?  
  Make sure you understand this before proceeding.

### 8.3. Clustering Using Dimensionality Reduction

In this section the IsMap and Laplacian Eigenmaps methods will be used for clustering of the _5 Rings_ data set.

For each data set:

1. Plot the Data Set  
   Plot the Data set in 3D.  
   **This is implemented**.
2. Reduce the Dimensionality of the Data  
   Reduce the dimensionality of the data to `d = 2` using each method.  
   Set the number of neighbors in the graph so the graph has a single component.
3. Apply the Clustering  
   Use K-Means for clustering with `k = 5`.  
4. Plot the Clustered Data  
   Plot the data with the clustering per method.  
   Plot the transformed labeled data and the original data per method.

* <font color='brown'>(**#**)</font> Pay attention to the difference in dimensions of the data to the derived Math formulations.
* <font color='brown'>(**#**)</font> The output should be 2 figures for each data set. You may show them in a single plot using sub plots.

In [ ]:
# Generate Data

mX = sp.io.loadmat('Rings5.mat')['pointCloud']
mX = mX.T

print(f'The data dimensions are {mX.shape[0]}x{mX.shape[1]}')

In [ ]:
# Plot Data
# Plotting the Data and a reference clustering by K-Means

K = 5 #<! Number of clusters

oKMeansCluster  = KMeans(n_clusters = K, n_init = 'auto')
vC              = oKMeansCluster.fit_predict(mX)

hF  = plt.figure(figsize = (12, 6))
hA1 = hF.add_subplot(1, 2, 1, projection = '3d')
hA2 = hF.add_subplot(1, 2, 2, projection = '3d')

Plot3DScatter(mX, hA1)
hA1.set_title('The 5 Rings Data')
hA1.view_init(elev = 45, azim = 300)

Plot3DScatter(mX, hA2, vC = vC)
hA2.set_title(f'The 5 Rings Data - Clustered by K-Means with K = {K}')
hA2.view_init(elev = 45, azim = 300)

plt.tight_layout()
plt.show()

In [ ]:
#===========================Fill This===========================#
# 1. Set parameters: `d`, `kNumNeighbors`, `σ` (Try to get a good clustering result).
# 2. Apply Dimensionality Reduction using IsoMap and Laplacian Eigen Maps.
# 3. Apply K-Means on the transformed data.
# 4. Display the clustered data in 2D (Low Dimension) and 3D (Original).
# !! You should use, in this case, the same number of neighbors for both algorithms.
# !! The output should be a figure of 2x2 axes (2D Scatter and 3D Scatter per method).
# !! You may use `Plot3DScatter()` and `Plot2DScatter()` for displaying the the data.

d               = 2
kNumNeighbors   = 50
σ               = 1

isomap = IsoMap(d, kNumNeighbors)
mZ_iso = isomap.fit_transform(mX)


lap = LaplacianEigenMaps(d, kNumNeighbors)
mZ_lap = lap.fit_transform(mX)



kmeans = KMeans(n_clusters=4)
labels_iso = kmeans.fit_predict(mZ_iso)
labels_lap = kmeans.fit_predict(mZ_lap)

# Create 2D scatter plots
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.scatter(mZ_iso[:, 0], mZ_iso[:, 1], c=labels_iso, cmap='viridis')
plt.title("IsoMap Dimensionality Reduction (2D)")
plt.xlabel("Dimension 1")
plt.ylabel("Dimension 2")

plt.subplot(1, 2, 2)
plt.scatter(mZ_lap[:, 0], mZ_lap[:, 1], c=labels_lap, cmap='viridis')
plt.title("Laplacian Dimensionality Reduction (2D)")
plt.xlabel("Dimension 1")
plt.ylabel("Dimension 2")

plt.show()

# Create 3D scatter plots (Original Data)
fig = plt.figure(figsize=(12, 6))

ax1 = fig.add_subplot(121, projection='3d')
ax1.scatter(mX[:, 0], mX[:, 1], mX[:, 2], c=labels_iso, cmap='viridis')
ax1.set_title("IsoMap Dimensionality Reduction (3D)")

ax2 = fig.add_subplot(122, projection='3d')
ax2.scatter(mX[:, 0], mX[:, 1], mX[:, 2], c=labels_lap, cmap='viridis')
ax2.set_title("Laplacian Dimensionality Reduction (3D)")

plt.show()
#===============================================================#

### 8.4. Question

In the above we used _Laplacian Eigenmaps_ for dimensionality reduction and then clustering.  
What would change if the task was to apply Spectral Clustering?  
Describe what will happen for the data above (The _5 Rings_).  
Address the changes needed in the implementation of the class `LaplcaianEigenMaps()` and the use of the class.

* <font color='brown'>(**#**)</font> You should use the ideas in _Question 5.2._ / _Question 5.3._.

### 8.4. Solution

<font color='red'>??? Fill the answer here ???</font>

---


In [ ]:
# Run Time
# Check Total Run Time.
# Don't change this!

endTime = time.time()

totalRunTime = endTime - startTime
print(f'Total Run Time: {totalRunTime} [Sec].')

if (totalRunTime > TOTAL_RUN_TIME):
    raise ValueError(f'You have exceeded the allowed run time as {totalRunTime} > {TOTAL_RUN_TIME}')